In [6]:
import config
import helper_fxns as hf
import transforms as tr
import importlib
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
%matplotlib inline

In [2]:
importlib.reload(hf)
importlib.reload(tr)
importlib.reload(config)
C = config.Config()

In [ ]:
gem_imgs("cyst", C.dims)

In [ ]:
def gen_imgs(cls, final_size):
    if cls == "cyst":
        imgs = gen_cysts(final_size)
    
    for cnt, img in enumerate(imgs):
        np.save(C.aug_dir + cls + "\\train_" + str(cnt), img)

# Artificial Cysts

In [5]:
[1,2,3]*2

[1, 2, 3, 1, 2, 3]

In [ ]:
def gen_cyst(final_size, n=50):
    imgs = []
    long_size_frac = [0.7, 0.9]
    min_side_ratio = 0.5
    
    obliqueness = list(np.linspace(min_side_ratio, 1, 3)) * math.ceil(n/
    sizes = np.linspace(long_size_frac[0], long_size_frac[1], num=n)

# Artificial HCC

# Artificial Hemangioma

# Artificial Cholangio

# Artificial Colorectal

# Artificial FNH